In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.parser import parse
from dotenv import dotenv_values
from pybit.unified_trading import HTTP

In [2]:
def open_long():
    while True:
        try:
            balance = float(session.get_wallet_balance(accountType="UNIFIED")['result']['list'][0]['coin'][0]['walletBalance'])
            price = float(session.get_tickers(category="linear",symbol="BTCUSDT")['result']['list'][0]['lastPrice'])
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)
    size = pos_pct * balance / price * leverage
    size = "{:.3f}".format(size)
    while True:
        try:
            session.place_order(category="linear", symbol="BTCUSDT", side="Buy", orderType="Market", qty=size,
                                stopLoss="{:.2f}".format(((1 - sl / leverage) * price)))
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)

In [3]:
def open_short():
    while True:
        try:
            balance = float(session.get_wallet_balance(accountType="UNIFIED")['result']['list'][0]['coin'][0]['walletBalance'])
            price = float(session.get_tickers(category="linear",symbol="BTCUSDT")['result']['list'][0]['lastPrice'])
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)
    size = pos_pct * balance / price * leverage
    size = "{:.3f}".format(size)
    while True:
        try:
            session.place_order(category="linear", symbol="BTCUSDT", side="Sell", orderType="Market", qty=size,
                                stopLoss="{:.2f}".format(((1 + sl / leverage) * price)))
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)

In [4]:
def close_long():
    while True:
        try:
            size = float(session.get_positions(category="linear", symbol="BTCUSDT")['result']['list'][0]['size'])
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)
    size = "{:.3f}".format(size)
    while True:
        try:
            session.place_order(category="linear", symbol="BTCUSDT", side="Sell", orderType="Market", qty=str(size),
                                reduceOnly=True)
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)

In [5]:
def close_short():
    while True:
        try:
            size = float(session.get_positions(category="linear", symbol="BTCUSDT")['result']['list'][0]['size'])
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)
    size = "{:.3f}".format(size)
    while True:
        try:
            session.place_order(category="linear", symbol="BTCUSDT", side="Buy", orderType="Market", qty=str(size),
                                reduceOnly=True)
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)

In [6]:
config = dotenv_values('.env')
API_KEY = config.get('BYBIT_TEST_API_KEY')
API_SECRET = config.get('BYBIT_TEST_API_SECRET')

In [7]:
session = HTTP(testnet=True, api_key=API_KEY, api_secret=API_SECRET)

In [8]:
# Params

tf = 60
w = 14
std_multiple = 3

sl = 0.1
leverage = 10
pos_pct = 0.9

In [9]:
assert int(session.get_positions(category="linear", symbol="BTCUSDT")['result']['list'][0]['leverage']) == leverage

# Trading

In [ ]:
first_candle = True

while True:
    try:
        candles = session.get_kline(category="linear", symbol="BTCUSDT", interval=tf, limit=w + 1)['result']['list']
        break
    except ConnectionError:
        print('Connection error. Trying again...')
        time.sleep(1)
last_w_candles = candles[1:]
curr_candle = candles[0]

last_w_candle_sizes = [float(x[2]) - float(x[3]) for x in last_w_candles]
last_w_candle_sizes_mean = np.mean(last_w_candle_sizes)
last_w_candle_sizes_std = np.std(last_w_candle_sizes, ddof=1)
thr = last_w_candle_sizes_mean + std_multiple * last_w_candle_sizes_std

curr_ts = int(curr_candle[0])
next_ts = curr_ts + 3600000

while True:
    while True:
        try:
            pos_size = float(session.get_positions(category="linear", symbol="BTCUSDT")['result']['list'][0]['size'])
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)
    if 1000 * time.time() >= next_ts:
        if pos_size > 0:
            print('Closing position...')
            while True:
                try:
                    side = session.get_positions(category="linear", symbol="BTCUSDT")['result']['list'][0]['side']
                    break
                except ConnectionError:
                    print('Connection error. Trying again...')
                    time.sleep(1)
            if side == 'Buy':
                close_long()
            elif side == 'Sell':
                close_short()
        print('Next candle...\n')
        first_candle = False
        
        while True:
            try:
                candles = session.get_kline(category="linear", symbol="BTCUSDT", interval=tf, limit=w + 1)['result']['list']
                break
            except ConnectionError:
                print('Connection error. Trying again...')
                time.sleep(1)
        last_w_candles = candles[1:]
        curr_candle = candles[0]
        last_w_candle_sizes = [float(x[2]) - float(x[3]) for x in last_w_candles]
        last_w_candle_sizes_mean = np.mean(last_w_candle_sizes)
        last_w_candle_sizes_std = np.std(last_w_candle_sizes, ddof=1)
        thr = last_w_candle_sizes_mean + std_multiple * last_w_candle_sizes_std
        curr_ts = int(curr_candle[0])
        next_ts = curr_ts + 3600000
    
    while True:
        try:
            curr_candle = session.get_kline(category="linear", symbol="BTCUSDT", interval=tf, limit=1)['result']['list'][0]
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)
    curr_candle_size = float(curr_candle[2]) - float(curr_candle[3])
    
    if not first_candle and (pos_size == 0) and (curr_candle_size >= thr):
        print(f'Current candle size ({curr_candle_size}) > threshold ({thr})')
        if float(curr_candle[4]) > float(curr_candle[1]):  # if close > open
            print('Opening short...')
            open_short()
        elif float(curr_candle[4]) < float(curr_candle[1]):  # if close < open
            print('Opening long...')
            open_long()
    
    time.sleep(1)